In [197]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [198]:
data = pd.read_csv('pretest6.csv', sep=';')
data.head()

,userid,gradeto,subfamilynameto,corporatetitleto,eventto,fired,fullpathto,deptto,topdeptto,regionto,agefake,classfake,groupfake
0,1000,15,Product Marketing,VP Product Marketing,Data Change,0,ProdMkt,Product Marketing,ProdMkt,Russia,77,1,A
1,10000,10,Software Engineering,Senior Developer,Data Change,0,FutureTechSecPlat,Secure Platform Development,FutureTech,Russia,77,1,A
2,10001,10,Software Engineering,Senior Research Developer,Transfer,0,R&DPDOLinuxKICSNetwIndPDK,KICS for Networks Industrial PDK,PDO,Russia,77,1,A
3,10002,7,Test Engineering R&D,Testing Engineer,Data Change,0,R&DIDOIntDevPlatform,Platform Development,IDO,Russia,77,1,A
4,10003,9,Software Engineering,Developer,Data Change,0,InIPA,Privacy Audit,InI,Russia,77,1,A


In [199]:
data.columns

Index(['userid', 'gradeto', 'subfamilynameto', 'corporatetitleto', 'eventto',
       'fired', 'fullpathto', 'deptto', 'topdeptto', 'regionto', 'agefake',
       'classfake', 'groupfake'],
      dtype='object')

In [200]:
cols_to_norm = ['userid','gradeto']

In [201]:
data[cols_to_norm] = data[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min() ))

In [202]:
feat_userid = tf.feature_column.numeric_column('userid')
feat_gradeto = tf.feature_column.numeric_column('gradeto')
feat_subfamilynameto = tf.feature_column.categorical_column_with_hash_bucket('subfamilynameto',hash_bucket_size=110)
feat_corporatetitleto = tf.feature_column.categorical_column_with_hash_bucket('corporatetitleto',hash_bucket_size=2050)
feat_eventto = tf.feature_column.categorical_column_with_hash_bucket('eventto',hash_bucket_size=25)
feat_fullpathto = tf.feature_column.categorical_column_with_hash_bucket('fullpathto',hash_bucket_size=2200)
feat_deptto = tf.feature_column.categorical_column_with_hash_bucket('deptto',hash_bucket_size=1450)
feat_topdeptto = tf.feature_column.categorical_column_with_hash_bucket('topdeptto',hash_bucket_size=150)
feat_regionto = tf.feature_column.categorical_column_with_hash_bucket('regionto',hash_bucket_size=20)
#feat_fired = tf.feature_column.numeric_column('fired')

In [203]:
feat_cols = [feat_userid,feat_gradeto,feat_subfamilynameto,feat_corporatetitleto,feat_eventto,feat_fullpathto,feat_deptto,feat_topdeptto,feat_regionto,]

In [204]:
x_data = data.drop('fired',axis=1) # надо понять зачем мы дропаем фичу, так как ниже возникает ошибка из-за этого
x_data.head()

,userid,gradeto,subfamilynameto,corporatetitleto,eventto,fullpathto,deptto,topdeptto,regionto,agefake,classfake,groupfake
0,0.000443,1.000000,Product Marketing,VP Product Marketing,Data Change,ProdMkt,Product Marketing,ProdMkt,Russia,77,1,A
1,0.997342,0.666667,Software Engineering,Senior Developer,Data Change,FutureTechSecPlat,Secure Platform Development,FutureTech,Russia,77,1,A
2,0.997452,0.666667,Software Engineering,Senior Research Developer,Transfer,R&DPDOLinuxKICSNetwIndPDK,KICS for Networks Industrial PDK,PDO,Russia,77,1,A
3,0.997563,0.466667,Test Engineering R&D,Testing Engineer,Data Change,R&DIDOIntDevPlatform,Platform Development,IDO,Russia,77,1,A
4,0.997674,0.600000,Software Engineering,Developer,Data Change,InIPA,Privacy Audit,InI,Russia,77,1,A


In [205]:
labels = data['fired']

In [206]:
X_train, X_test, y_train, y_test = train_test_split(x_data,labels, test_size=0.33, random_state=101)

In [207]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=1000,shuffle=True)

In [208]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Alex\\AppData\\Local\\Temp\\tmp_w37ivpb', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000015A11DEA080>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [209]:
model.train(input_fn=input_func,steps=1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Alex\AppData\Local\Temp\tmp_w37ivpb\model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 236.48
INFO:tensorflow:loss = 2.2227178, step = 101 (0.424 sec)
INFO:tensorflow:global_step/sec: 466.361
INFO:tensorflow:loss = 1.2413304, step = 201 (0.214 sec)
INFO:tensorflow:global_step/sec: 456.785
INFO:tensorflow:loss = 0.6820797, step = 301 (0.219 sec)
INFO:tensorflow:global_step/sec: 516.841
INFO:tensorflow:loss = 0.56318355, step = 401 (0.193 sec)
INFO:tensorflow:Saving checkpoints for 450 into C:\Users\Alex\AppData\Local\Temp\tmp_w37ivpb\model.ckpt.
INFO:tensorflow:Loss for final step: 0.8612195.


In [210]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,y=y_test,batch_size=10,num_epochs=1,shuffle=False)

In [211]:
results = model.evaluate(eval_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-03-18:55:07
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Alex\AppData\Local\Temp\tmp_w37ivpb\model.ckpt-450
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-03-18:55:07
INFO:tensorflow:Saving dict for global step 450: accuracy = 1.0, accuracy_baseline = 0.82608694, auc = 0.9999999, auc_precision_recall = 0.99999976, average_loss = 0.07104585, global_step = 450, label/mean = 0.17391305, loss = 0.5446849, precision = 1.0, prediction/mean = 0.15952869, recall = 1.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 450: C:\Users\Alex\AppData\Local\Temp\tmp_w37ivpb\model.ckpt-450


In [212]:
results

{'accuracy': 1.0,
 'accuracy_baseline': 0.82608694,
 'auc': 0.9999999,
 'auc_precision_recall': 0.99999976,
 'average_loss': 0.07104585,
 'label/mean': 0.17391305,
 'loss': 0.5446849,
 'precision': 1.0,
 'prediction/mean': 0.15952869,
 'recall': 1.0,
 'global_step': 450}

In [213]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=10,num_epochs=1,shuffle=False)

In [214]:
predictions = model.predict(pred_input_func)

In [215]:
my_pred = list(predictions)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Alex\AppData\Local\Temp\tmp_w37ivpb\model.ckpt-450
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [216]:
my_pred

[{'logits': array([-4.2500477], dtype=float32),
  'logistic': array([0.01406297], dtype=float32),
  'probabilities': array([0.98593706, 0.01406297], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([0.04572296], dtype=float32),
  'logistic': array([0.5114288], dtype=float32),
  'probabilities': array([0.4885713, 0.5114288], dtype=float32),
  'class_ids': array([1], dtype=int64),
  'classes': array([b'1'], dtype=object)},
 {'logits': array([-3.7140894], dtype=float32),
  'logistic': array([0.0237975], dtype=float32),
  'probabilities': array([0.97620255, 0.0237975 ], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object)},
 {'logits': array([-4.6239433], dtype=float32),
  'logistic': array([0.00971864], dtype=float32),
  'probabilities': array([0.99028134, 0.00971864], dtype=float32),
  'class_ids': array([0], dtype=int64),
  'classes': array([b'0'], dtype=object)},
 {'logits': 